<a href="https://colab.research.google.com/github/NiyazovIlia/natural_language/blob/lesson_3/NLP_lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.jsonl

--2022-08-31 10:37:12--  https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.jsonl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/43l702z5a5i2w8j/gazeta_train.jsonl [following]
--2022-08-31 10:37:12--  https://www.dropbox.com/s/raw/43l702z5a5i2w8j/gazeta_train.jsonl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf15409f97bc98243bbcc0523f7.dl.dropboxusercontent.com/cd/0/inline/BsAgEe27qMw0QRXNzzABxQZl4j953ecCXQxLgfAs2TOt-jCx1RkslljxWwO9qLkie3E_prX8ue3Dwss0IwL7XDDgmu7k1FSyD0nZWOqJZ1QcTrZdBQT1wPaiNQgL08hgdxrQZ2-Tv8qEzzA4rBckpqv8_-s_FUVUqcgixf9IleWMPA/file# [following]
--2022-08-31 10:37:13--  https://ucf15409f97bc98243bbcc0523f7.dl.dropboxusercontent.com/cd/0/inline/BsAgEe27qMw0QRXNzzABxQZl4j953ecCXQxLgfAs2TOt-jCx1RkslljxWwO9qL

In [ ]:
emoticon_dict = {
":)": "счастливый",
":‑)": "счастливый",
":-]": "счастливый",
":-3": "счастливый",
":->": "счастливый",
"8-)": "счастливый",
":-}": "счастливый",
":o)": "счастливый",
":c)": "счастливый",
":^)": "счастливый",
"=]": "счастливый",
"=)": "счастливый",
"<3": "счастливый",
":-(": "грустный",
":(": "грустный",
":c": "грустный",
":<": "грустный",
":[": "грустный",
">:[": "грустный",
":{": "грустный",
">:(": "грустный",
":-c": "грустный",
":-< ": "грустный",
":-[": "грустный",
":-||": "грустный"
}

In [ ]:
!pip install pymorphy2
!pip install annoy
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import annoy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim.downloader as ap
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm_notebook


import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import tokenize as tknz
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import pandas as pd

train = pd.read_json('gazeta_train.jsonl', lines=True)

In [ ]:
train

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01
...,...,...,...,...,...
52395,https://www.gazeta.ru/social/2010/10/13/342826...,Следователя следственного отдела московского р...,Взятку привезли на «Мерседесе»,В Москве за вымогательство $700 тысяч задержан...,2010-10-13 14:01:03
52396,https://www.gazeta.ru/social/2011/01/24/350307...,В понедельник из-за взрыва в Домодедово в оста...,Аэропорты вызывают на досмотр,Сразу после теракта в московском аэропорту Дом...,2011-01-24 23:30:01
52397,https://www.gazeta.ru/culture/2014/05/19/a_603...,В Лас-Вегасе прошла церемония награждения побе...,Тимберлейк не увидел Майкла Джексона,Вручены награды музыкального журнала Billboard...,2014-05-19 15:02:31
52398,https://www.gazeta.ru/social/2013/08/21/560047...,В среду Абинский райсуд Краснодарского края пр...,Крымские чиновники уедут на поселение,Абинский райсуд Краснодарского края вынес приг...,2013-08-21 15:14:43


In [ ]:
df = pd.DataFrame(train['summary'])

In [ ]:
df

,summary
0,"В 2011 году из России уйдет $80 млрд, считают ..."
1,"Юлия Соловьева, экс-директор холдинга «Профмед..."
2,Суд арестовал на два месяца четверых экс-чинов...
3,Мнения предпринимателей по поводу вступления в...
4,На Первом канале завершился третий сезон шоу «...
...,...
52395,В Москве за вымогательство $700 тысяч задержан...
52396,Сразу после теракта в московском аэропорту Дом...
52397,Вручены награды музыкального журнала Billboard...
52398,Абинский райсуд Краснодарского края вынес приг...


In [ ]:
df['preprocessed'] = df.summary

In [ ]:
df

,summary,preprocessed
0,"В 2011 году из России уйдет $80 млрд, считают ...","В 2011 году из России уйдет $80 млрд, считают ..."
1,"Юлия Соловьева, экс-директор холдинга «Профмед...","Юлия Соловьева, экс-директор холдинга «Профмед..."
2,Суд арестовал на два месяца четверых экс-чинов...,Суд арестовал на два месяца четверых экс-чинов...
3,Мнения предпринимателей по поводу вступления в...,Мнения предпринимателей по поводу вступления в...
4,На Первом канале завершился третий сезон шоу «...,На Первом канале завершился третий сезон шоу «...
...,...,...
52395,В Москве за вымогательство $700 тысяч задержан...,В Москве за вымогательство $700 тысяч задержан...
52396,Сразу после теракта в московском аэропорту Дом...,Сразу после теракта в московском аэропорту Дом...
52397,Вручены награды музыкального журнала Billboard...,Вручены награды музыкального журнала Billboard...
52398,Абинский райсуд Краснодарского края вынес приг...,Абинский райсуд Краснодарского края вынес приг...


In [ ]:
def tweets_preprocessing(example):
    # Заменим на нижний регистр
    example = example.lower()
    # Заменим пунктуацию на пробелы
    example = re.sub(r'[^\w\s]', ' ', example)
    # Заменим спец. символы на пробелы
    example = re.sub(r'[^a-zA-Zа-яёA-ЯЁ0-9]', ' ', example)
    # Заменим числа на пробелы
    example = re.sub(r'[^a-zA-Zа-яёA-ЯЁ]', ' ', example)
    # Удалим символы
    example = ' '.join([w for w in example.split() if len(w)>1])
    # Удалим из текста слова длиной в 2 символа
    example = [w for w in example.split() if len(w)>2]
    # Удалим стоп-слова
    stop_words = get_stop_words("ru") + stopwords.words('russian')
    example = [w for w in example if w not in stop_words] 
    # Приведем к нормальной форме
    stemmer = WordNetLemmatizer()
    test = []
    for i in example:
      test.append(stemmer.lemmatize(i))
    
    return test

In [ ]:
df['preprocessed'] = df.preprocessed.apply(tweets_preprocessing)
df

,summary,preprocessed
0,"В 2011 году из России уйдет $80 млрд, считают ...","[россии, уйдет, млрд, считают, минэкономразвит..."
1,"Юлия Соловьева, экс-директор холдинга «Профмед...","[юлия, соловьева, экс, директор, холдинга, про..."
2,Суд арестовал на два месяца четверых экс-чинов...,"[суд, арестовал, месяца, четверых, экс, чиновн..."
3,Мнения предпринимателей по поводу вступления в...,"[мнения, предпринимателей, поводу, вступления,..."
4,На Первом канале завершился третий сезон шоу «...,"[первом, канале, завершился, сезон, шоу, голос..."
...,...,...
52395,В Москве за вымогательство $700 тысяч задержан...,"[москве, вымогательство, задержан, следователь..."
52396,Сразу после теракта в московском аэропорту Дом...,"[сразу, теракта, московском, аэропорту, домоде..."
52397,Вручены награды музыкального журнала Billboard...,"[вручены, награды, музыкального, журнала, bill..."
52398,Абинский райсуд Краснодарского края вынес приг...,"[абинский, райсуд, краснодарского, края, вынес..."


In [ ]:
text = [tweet for tweet in df['preprocessed'].values if len(tweet)>2]
text[:3]

[['россии',
  'уйдет',
  'млрд',
  'считают',
  'минэкономразвития',
  'месяца',
  'центробанк',
  'давал',
  'оценку',
  'млрд',
  'повысив',
  'первоначальный',
  'прогноз',
  'вдвое',
  'отток',
  'капитала',
  'страны',
  'усиливается',
  'кризиса',
  'европе',
  'декабре',
  'российским',
  'компаниям',
  'выплачивать',
  'внешние',
  'долги',
  'движение',
  'капитала',
  'повлияли',
  'выборы',
  'несмотря',
  'предсказуемость',
  'бизнес',
  'хочет',
  'ясности',
  'перестановками',
  'правительстве'],
 ['юлия',
  'соловьева',
  'экс',
  'директор',
  'холдинга',
  'профмедиа',
  'возглавила',
  'российский',
  'офис',
  'google',
  'поиск',
  'американская',
  'интернет',
  'корпорация',
  'вела',
  'полгода',
  'медиаменеджер',
  'опытом',
  'работы',
  'международных',
  'компаниях',
  'найдет',
  'общий',
  'язык',
  'акционерами',
  'прорыва',
  'ждать',
  'технического',
  'специалиста',
  'маркетолога',
  'говорят',
  'аналитики'],
 ['суд',
  'арестовал',
  'месяца',
  '

In [ ]:
modelW2V = Word2Vec(sentences=text, size=300, window=5, min_count=1)
modelFT = FastText(sentences=text, size=300, min_count=1, window=5, workers=8)

In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in df['preprocessed']:
    n_w2v = 0
    n_ft = 0
#     spls = line.split("\t")
    index_map[counter] = line

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in line:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [ ]:
def get_response(tweet, index, model, index_map):
    tweet = tweets_preprocessing(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [df.summary.values[i] for i in answers]

In [ ]:
test_tweet = 'Какая ужасная погода на улице'

In [ ]:
get_response(test_tweet, w2v_index, modelW2V, index_map)

['В Москве и Подмосковье продолжается похолодание. По ночам ожидаются морозы, будет идти снег, заметут метели. К концу предстоящей недели столичный регион уже покроет снегом.',
 'В выходные в Москве ожидается потепление до +18° С, но оно будет недолгим. Уже в понедельник температура понизится, пойдут дожди и погода вернется к климатической норме первой половины октября. Ночами возможны заморозки.',
 'С приходом весны в столичный регион придет и настоящая весенняя погода. На этих выходных в Москве еще будет морозно, а с наступлением будней начнет постепенно теплеть — к середине следующей недели установится плюсовая температура. К следующему уикенду погода в Москве будет соответствовать климатической норме начала марта.',
 'Мокрый снег и холодный ветер задержатся в Москве еще на пару дней. По-настоящему весенняя погода ждет горожан в начале следующей недели. Днем столбики термометров поднимутся до отметки в +8°С. Снег начнет интенсивно таять, осадки прекратятся, а на улице будет преимуще

In [ ]:
get_response(test_tweet, ft_index, modelFT, index_map)

['Попавшая под колеса злостного лихача на BMW и потерявшая ребенка на девятом месяце беременности москвичка скончалась в больнице. Врачи неделю боролись за жизнь женщины, но не смогли ее спасти. Вторая пострадавшая находится в стабильно тяжелом состоянии. Женщины ждали на переходе зеленого сигнала, но их сбила вылетевшая на тротуар иномарка. При этом близкие погибшей уверяют, что водитель от медосвидетельствования отказался, а за год накопил 49 штрафов ГИБДД.',
 'Шесть человек погибли и около 25 пострадали в результате ДТП с участием рейсового автобуса в Подмосковье. Водитель на скользкой дороге потерял управление, после чего многотонная машина улетела в кювет, перевернувшись на бок. Это уже вторая крупная авария с участием автобуса за последние сутки — в ночь на 6 октября недалеко от границ Подмосковья в смертельное ДТП попал автобус с гражданами Узбекистана.',
 'Пять человек погибли в крупной аварии под Томском: при столкновении «Газель» снесла всю верхнюю часть «Жигулей», люди сконч

**Как видим modelW2V работает гораздо лучше**